<a href="https://colab.research.google.com/github/aqilanwar/meal-planner-using-ga/blob/main/GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import secrets
import collections
import random
import os
import pandas as pd


url = 'https://raw.githubusercontent.com/aqilanwar/meal-planner-using-ga/main/food-dataset.csv'
df = pd.read_csv(url)

df['Calories'] = pd.to_numeric(df['Calories'])

Menu = collections.namedtuple(
    'Menu', ['Name', 'Protein', 'Fat', 'Carbohydrates', 'Calories'])



# Type of food : Fruit, Protein, Vegetable, Dairy products

Dairy = []
Protein = []
Vegetable = []
Fruit = []

# Store from dataframe to list of each type of food.
for ind in df.index:
  if df['Category'][ind] == 'Dairy products':
    Dairy.append(Menu(df['Food'][ind], df['Protein'][ind] ,df['Fat'][ind] ,df['Carbs'][ind],df['Calories'][ind]))

  elif df['Category'][ind] == 'Protein':
    Protein.append(Menu(df['Food'][ind], df['Protein'][ind] ,df['Fat'][ind] ,df['Carbs'][ind],df['Calories'][ind]))

  elif df['Category'][ind] == 'Vegetables':
    Vegetable.append(Menu(df['Food'][ind], df['Protein'][ind] ,df['Fat'][ind] ,df['Carbs'][ind],df['Calories'][ind]))

  elif df['Category'][ind] == 'Fruits':
    Fruit.append(Menu(df['Food'][ind], df['Protein'][ind] ,df['Fat'][ind] ,df['Carbs'][ind],df['Calories'][ind]))
  
def generate_menuSatuHari():
    menu_satuHari = []

    genome_dairy = secrets.choice(Dairy)
    genone_protein = secrets.choice(Protein)
    genome_vegetable = secrets.choice(Vegetable)
    genome_fruit = secrets.choice(Fruit)

    menu_satuHari.insert(0, genome_dairy)
    menu_satuHari.insert(1, genone_protein)
    menu_satuHari.insert(2, genome_vegetable)
    menu_satuHari.insert(3, genome_fruit)

    return menu_satuHari


def generate_genome(size: int):
    genome = []
    for i in range(0, size):
        for j in range(0, 3):
            menu_satuHari = []

            genome_dairy = secrets.choice(Dairy)
            genone_protein = secrets.choice(Protein)
            genome_vegetable = secrets.choice(Vegetable)
            genome_fruit = secrets.choice(Fruit)

            menu_satuHari.insert(0, genome_dairy)
            menu_satuHari.insert(1, genone_protein)
            menu_satuHari.insert(2, genome_vegetable)
            menu_satuHari.insert(3, genome_fruit)

        genome.insert(j, menu_satuHari)
    return genome


def generate_population(size: int):
    population = []
    for i in range(size):
        population.insert(i, generate_genome(21))  # 21 = 7 hari * 3 kali makan
    return population


def generate_fitness(limit_kalori: int, population: list):
    fitness = []
    idx = 0

    for genome in population:
        fitness_pct = 0
        kalori = 0

        for hari in (genome):
            for i in hari:
                kalori += i.Calories

        fitness_pct = (kalori/limit_kalori) * 100
        if (kalori > limit_kalori):
            fitness.insert(idx, 0)
        else:
            fitness.insert(idx, fitness_pct)
        idx += 1

    return fitness


def selection(fitness: list):

    fitness_clone = fitness[:]

    parent1 = max(fitness_clone, key=float)
    parent1_index = fitness.index(parent1)

    fitness_clone.pop(parent1_index)

    parent2 = max(fitness_clone, key=float)
    parent2_index = fitness.index(parent2)

    return [parent1_index, parent2_index]


def crossover(parent1: list, parent2: list):
    child1 = parent1[:]
    child2 = parent2[:]

    crossover_point = round(len(child1)/2)

    child1[0:crossover_point] = parent2[0:crossover_point]
    child2[0:crossover_point] = parent1[0:crossover_point]

    return [child1, child2]


def mutation(child: list, mutation_rate: int):
    mutant = child[:]

    for i in range(len(mutant)):
        if(random.uniform(0, 1) <= mutation_rate):
            mutant[i] = generate_menuSatuHari()

    return mutant


def regeneration(population: list, fitness: list, mutant1: list, mutant2: list):
    min_selection1 = min(fitness, key=float)
    min_selection1_index = fitness.index(min_selection1)

    fitness.pop(min_selection1_index)
    population.pop(min_selection1_index)

    min_selection2 = min(fitness, key=float)
    min_selection2_index = fitness.index(min_selection2)

    fitness.pop(min_selection2_index)
    population.pop(min_selection2_index)

    population.append(mutant1)
    population.append(mutant2)

    return population


def main():


    print(Menu)
    
    limit_kalori = (int(input("Please enter your BMR : ")) * 7) 
    mutation_rate = 0.2
    isLooping = True

    population = generate_population(10)
    fitness = generate_fitness(limit_kalori, population)

    generation = 0
    while(isLooping):
        parents = selection(fitness)
        parent1 = population[parents[0]]
        parent2 = population[parents[1]]

        [child1, child2] = crossover(parent1, parent2)

        mutant1 = mutation(child1, mutation_rate)
        mutant2 = mutation(child2, mutation_rate)

        population = regeneration(population, fitness, mutant1, mutant2)
        fitness = generate_fitness(limit_kalori, population)

        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"Best Fitness: {max(fitness)}")

        generation += 1
        if(max(fitness) == 100):
            isLooping = False
            population_index = fitness.index(max(fitness))
            kalori_sementara = 0

            print(f"Best solution found at {generation} generations!")
            print(f"=========================")

            # Show the list of menu suggested 
            for i in population[population_index]:
                print(i)
                for j in i:
                    kalori_sementara += j.kalori

            print(kalori_sementara)


if __name__ == "__main__":
    main()

Please enter your BMR : 1560
Best Fitness: 0
Best Fitness: 99.47802197802199
Best Fitness: 99.47802197802199
Best Fitness: 99.47802197802199
Best Fitness: 99.47802197802199
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.98168498168498
Best Fitness: 99.9816849816